In [1]:
#Arbol - Librerias
!conda install -c etetoolkit ete3
!export PATH=~/anaconda_ete/bin:$PATH
!conda install -c bioconda iqtree
!conda install -c anaconda pillow
!apt-get install -y clustalo

#Mapa - librerias
!pip install geopy
!pip install folium
!pip install selenium
!pip install cefpython3
!pip install biopython


#Ventanas
!apt-get install -y python-tk
!pip install tkinterhtml

Solving environment: | 
  - anaconda/linux-64::ca-certificates-2020.6.24-0, anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::conda-4.8.3-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0
  - anaconda/linux-64::ca-certificates-2020.6.24-0, anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::conda-4.8.3-py37_0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::conda-4.8.3-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca-certificates-2020.6.24-0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::openssl-1.1.1g-h7b6447c_0, defaults/linux-64::ca-certificates-2020.6.24-0, defaults/linux-64::conda-4.8.3-py37_0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, anaconda/linux-64::conda-4.8.3-py37_0, defaults/linux-64::ca-certificates-2020.6.24-0, defaults/linux-64::openssl-1.1.1g-h7b6447c_0
  - anaconda/linux-64::certifi-2020.6.20-py37_0, defaults/linux-64::ca-

In [5]:
from cefpython3 import cefpython as cef
import ctypes
try:
    import tkinter as tk
except ImportError:
    import Tkinter as tk
import sys
import os
import platform
import logging as _logging

WindowUtils = cef.WindowUtils()
WINDOWS = (platform.system() == "Windows")
LINUX = (platform.system() == "Linux")
MAC = (platform.system() == "Darwin")
logger = _logging.getLogger("tkinter_.py")
IMAGE_EXT = ".png" if tk.TkVersion > 8.5 else ".gif"

def main():
    print("SE EJECUTA ESTO GUACHO")
    logger.setLevel(_logging.INFO)
    stream_handler = _logging.StreamHandler()
    formatter = _logging.Formatter("[%(filename)s] %(message)s")
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)
    logger.info("CEF Python {ver}".format(ver=cef.__version__))
    logger.info("Python {ver} {arch}".format(
            ver=platform.python_version(), arch=platform.architecture()[0]))
    logger.info("Tk {ver}".format(ver=tk.Tcl().eval('info patchlevel')))
    assert cef.__version__ >= "55.3", "CEF Python v55.3+ required to run this"
    sys.excepthook = cef.ExceptHook  
    root = tk.Tk()
    app = MainFrame(root)
    cef.Initialize()
    app.mainloop()
    cef.Shutdown()


class MainFrame(tk.Frame):

    def __init__(self, root):
        self.browser_frame = None
        self.navigation_bar = None
        root.geometry("900x640")
        tk.Grid.rowconfigure(root, 0, weight=1)
        tk.Grid.columnconfigure(root, 0, weight=1)
        tk.Frame.__init__(self, root)
        self.master.title("Llego el mapa, bitches")
        self.master.protocol("WM_DELETE_WINDOW", self.on_close)
        self.master.bind("<Configure>", self.on_root_configure)
        self.bind("<Configure>", self.on_configure)
        self.bind("<FocusIn>", self.on_focus_in)
        self.bind("<FocusOut>", self.on_focus_out)
        tk.Grid.rowconfigure(self, 0, weight=0)
        tk.Grid.columnconfigure(self, 0, weight=0)
        self.browser_frame = BrowserFrame(self, self.navigation_bar)
        self.browser_frame.grid(row=1, column=0,
                                sticky=(tk.N + tk.S + tk.E + tk.W))
        tk.Grid.rowconfigure(self, 1, weight=1)
        tk.Grid.columnconfigure(self, 0, weight=1)
        self.pack(fill=tk.BOTH, expand=tk.YES)

    def on_root_configure(self, _):
        logger.debug("MainFrame.on_root_configure")
        if self.browser_frame:
            self.browser_frame.on_root_configure()

    def on_configure(self, event):
        logger.debug("MainFrame.on_configure")
        if self.browser_frame:
            width = event.width
            height = event.height
            if self.navigation_bar:
                height = height - self.navigation_bar.winfo_height()
            self.browser_frame.on_mainframe_configure(width, height)

    def on_focus_in(self, _):
        logger.debug("MainFrame.on_focus_in")

    def on_focus_out(self, _):
        logger.debug("MainFrame.on_focus_out")

    def on_close(self):
        if self.browser_frame:
            self.browser_frame.on_root_close()
        self.master.destroy()

    def get_browser(self):
        if self.browser_frame:
            return self.browser_frame.browser
        return None

    def get_browser_frame(self):
        if self.browser_frame:
            return self.browser_frame
        return None

    def setup_icon(self):
        resources = os.path.join(os.path.dirname(__file__), "resources")
        icon_path = os.path.join(resources, "tkinter"+IMAGE_EXT)
        if os.path.exists(icon_path):
            self.icon = tk.PhotoImage(file=icon_path)
            self.master.call("wm", "iconphoto", self.master._w, self.icon)


class BrowserFrame(tk.Frame):

    def __init__(self, master, navigation_bar=None):
        self.navigation_bar = navigation_bar
        self.closing = False
        self.browser = None
        tk.Frame.__init__(self, master)
        self.bind("<FocusIn>", self.on_focus_in)
        self.bind("<FocusOut>", self.on_focus_out)
        self.bind("<Configure>", self.on_configure)
        self.focus_set()

    def embed_browser(self):
        window_info = cef.WindowInfo()
        rect = [0, 0, self.winfo_width(), self.winfo_height()]
        window_info.SetAsChild(self.get_window_handle(), rect)
        #Ubicacion absoluta del archivo
        self.browser = cef.CreateBrowserSync(window_info,
                                             url="file:///home/chesire/Escritorio/BiocambioFacundo/Bioinformatica-UNQ/TP%20-%20FINAL/Mapa.html")
        assert self.browser
        self.browser.SetClientHandler(LoadHandler(self))
        self.browser.SetClientHandler(FocusHandler(self))
        self.message_loop_work()

    def get_window_handle(self):
        if self.winfo_id() > 0:
            return self.winfo_id()
        elif MAC:
            from AppKit import NSApp
            import objc
            return objc.pyobjc_id(NSApp.windows()[-1].contentView())
        else:
            raise Exception("Couldn't obtain window handle")

    def message_loop_work(self):
        cef.MessageLoopWork()
        self.after(10, self.message_loop_work)

    def on_configure(self, _):
        if not self.browser:
            self.embed_browser()

    def on_root_configure(self):
        if self.browser:
            self.browser.NotifyMoveOrResizeStarted()

    def on_mainframe_configure(self, width, height):
        if self.browser:
            if WINDOWS:
                ctypes.windll.user32.SetWindowPos(
                    self.browser.GetWindowHandle(), 0,
                    0, 0, width, height, 0x0002)
            elif LINUX:
                self.browser.SetBounds(0, 0, width, height)
            self.browser.NotifyMoveOrResizeStarted()

    def on_focus_in(self, _):
        logger.debug("BrowserFrame.on_focus_in")
        if self.browser:
            self.browser.SetFocus(True)

    def on_focus_out(self, _):
        logger.debug("BrowserFrame.on_focus_out")
        if LINUX and self.browser:
            self.browser.SetFocus(False)

    def on_root_close(self):
        if self.browser:
            self.browser.CloseBrowser(True)
            self.clear_browser_references()
        self.destroy()

    def clear_browser_references(self):
        self.browser = None


class LoadHandler(object):

    def __init__(self, browser_frame):
        self.browser_frame = browser_frame

    def OnLoadStart(self, browser, **_):
        pass
        
class FocusHandler(object):
    
    def __init__(self, browser_frame):
        self.browser_frame = browser_frame

    def OnTakeFocus(self, next_component, **_):
        logger.debug("FocusHandler.OnTakeFocus, next={next}"
                     .format(next=next_component))

    def OnSetFocus(self, source, **_):
        logger.debug("FocusHandler.OnSetFocus, source={source}"
                     .format(source=source))
        if LINUX:
            return False
        else:
            return True

    def OnGotFocus(self, **_):
        logger.debug("FocusHandler.OnGotFocus")
        if LINUX:
            self.browser_frame.focus_set()


class NavigationBar(tk.Frame):

    def __init__(self, master):
        self.back_state = tk.NONE
        self.forward_state = tk.NONE
        self.back_image = None
        self.forward_image = None
        self.reload_image = None

        tk.Frame.__init__(self, master)
        resources = os.path.join(os.path.dirname(__file__), "resources")

        back_png = os.path.join(resources, "back"+IMAGE_EXT)
        if os.path.exists(back_png):
            self.back_image = tk.PhotoImage(file=back_png)

        forward_png = os.path.join(resources, "forward"+IMAGE_EXT)
        if os.path.exists(forward_png):
            self.forward_image = tk.PhotoImage(file=forward_png)

        reload_png = os.path.join(resources, "reload"+IMAGE_EXT)
        if os.path.exists(reload_png):
            self.reload_image = tk.PhotoImage(file=reload_png)

            self.url_entry = tk.Entry(self)
        self.url_entry.bind("<FocusIn>", self.on_url_focus_in)
        self.url_entry.bind("<FocusOut>", self.on_url_focus_out)
        self.url_entry.bind("<Return>", self.on_load_url)
        self.url_entry.bind("<Button-1>", self.on_button1)
        self.url_entry.grid(row=0, column=3,
                            sticky=(tk.N + tk.S + tk.E + tk.W))
        tk.Grid.rowconfigure(self, 0, weight=100)
        tk.Grid.columnconfigure(self, 3, weight=100)

        self.update_state()

    def go_back(self):
        if self.master.get_browser():
            self.master.get_browser().GoBack()

    def go_forward(self):
        if self.master.get_browser():
            self.master.get_browser().GoForward()

    def reload(self):
        if self.master.get_browser():
            self.master.get_browser().Reload()

    def set_url(self, url):
        self.url_entry.delete(0, tk.END)
        self.url_entry.insert(0, url)

    def on_url_focus_in(self, _):
        logger.debug("NavigationBar.on_url_focus_in")

    def on_url_focus_out(self, _):
        logger.debug("NavigationBar.on_url_focus_out")

    def on_load_url(self, _):
        if self.master.get_browser():
            self.master.get_browser().StopLoad()
            self.master.get_browser().LoadUrl(self.url_entry.get())

    def on_button1(self, _):
        logger.debug("NavigationBar.on_button1")
        self.master.master.focus_force()

    def update_state(self):
        pass

class Tabs(tk.Frame):

    def __init__(self):
        tk.Frame.__init__(self)

class MapaWindows():

    def render(self):
        main()

map = MapaWindows()

OSError: /home/chesire/anaconda3/bin/../lib/libgio-2.0.so.0: undefined symbol: g_ref_count_inc

In [6]:
from PIL import Image, ImageTk
from tkinter import PhotoImage, messagebox
from clases.Arbol import Arbol
from clases.Mapa import Mapa, NotUbicacion, UbicacionNotFound
from clases.Uploader import Uploader
from clases.Alineador import Alineador
from tkinterhtml import HtmlFrame
import tkinter as tk

class Handler():
    def __init__(self):
        self.flag_map = False
        self.flag_upload = False
        self.flag_align = False
        self.uploader = Uploader()
        
    def client_exit(self):
        root.destroy()
        
    def upload_fasta(self):        
        if self.uploader.loadFasta():
            self.flag_upload = True
            messagebox.showinfo(message="Se cargó el FASTA correctamente!", title="")
        else:
            messagebox.showinfo(message="No se cargó el FASTA", title="")
            
    def align_secuences(self):
        if self.flag_upload:
            alineador = Alineador()
            alineador.alinear()
            #Aca iria el loading
            self.flag_align = True
            messagebox.showinfo(message="Alineado mostro!", title="")
        else:
            messagebox.showinfo(message="Carga primero el FASTA, titán", title="")
        
        
    def make_tree(self):
        if self.flag_upload and self.flag_align:
            arbol = Arbol()
            arbol.armarArbol()
        else:
            messagebox.showinfo(message="Primero alinea, mostro", title="")
        
    def make_map(self):
        mapa = Mapa()
        if not self.flag_upload:# or not self.flag_align:
            messagebox.showinfo(message="Y si primero cargas el fasta o alineas?", title="")
        else:
            try:
                if not self.flag_map:
                    self.flag_map = True
                    mapita = mapa.armar_mapa(self.uploader.ubicaciones).render_mapa()
                    mapa.guardar_mapa()
                    MapaWindows().render()
                    self.flag_map = False
            except UbicacionNotFound as e:
                messagebox.showinfo("Ojo al piojo", str(e))
            except NotUbicacion as e2:
                messagebox.showinfo("Ojo al piojo", str(e2))
            

class BkgrFrame(tk.Frame):
    def __init__(self, parent, file_path, width, height):
        super(BkgrFrame, self).__init__(parent, borderwidth=0, highlightthickness=0)

        self.canvas = tk.Canvas(self, width=width, height=height)
        self.canvas.pack()

        pil_img = Image.open(file_path)
        self.img = ImageTk.PhotoImage(pil_img.resize((width, height), Image.ANTIALIAS))
        self.bg = self.canvas.create_image(0, 0, anchor=tk.NW, image=self.img)

    def add(self, widget, x, y):
        canvas_window = self.canvas.create_window(x, y, anchor=tk.NW, window=widget)
        return widget


if __name__ == '__main__':

    IMAGE_PATH = './img/MenuHerramientaBio.png'
    WIDTH, HEIGTH = 800, 600

    root = tk.Tk()
    root.geometry('{}x{}'.format(WIDTH, HEIGTH))
    
    handler = Handler()

    bkrgframe = BkgrFrame(root, IMAGE_PATH, WIDTH, HEIGTH)
    bkrgframe.pack()
    
    exitPhot = PhotoImage(file = r"./img/BotonSalir.png")
    fastaPhot = PhotoImage(file = r"./img/BotonFasta.png")
    alignPhot = PhotoImage(file = r"./img/BotonAlinear.png")
    treePhot = PhotoImage(file = r"./img/BotonArbolear.png")
    mapPhot = PhotoImage(file = r"./img/BotonMapear.png")

    button1 = bkrgframe.add(tk.Button(root, image=exitPhot, command= handler.client_exit), 650, 8)
    button2 = bkrgframe.add(tk.Button(root, image=fastaPhot, command= handler.upload_fasta), 310, 180)
    button3 = bkrgframe.add(tk.Button(root, image=alignPhot, command= handler.align_secuences), 65, 340)
    button4 = bkrgframe.add(tk.Button(root, image=treePhot, command= handler.make_tree), 65, 430)
    button5 = bkrgframe.add(tk.Button(root, image=mapPhot, command= handler.make_map), 60, 520)

    root.resizable(width=False, height=False)
    root.wm_attributes('-type', 'splash')
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/chesire/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-6-4943f9146eae>", line 55, in make_map
    MapaWindows().render()
NameError: name 'MapaWindows' is not defined
